# Comparativa de Reducción de Ruido: NLM vs BM3D
Este notebook realiza una comparativa visual de dos de los algoritmos más potentes para la eliminación de ruido en imágenes: **Non-Local Means (NLM)** y **Block-Matching and 3D Filtering (BM3D)**.

En el framework VCF, estos filtros se aplican durante la etapa de **decodificación**, procesando la imagen una vez que ha sido reconstruida por el pipeline de compresión.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import display, Image

# Configuración de niveles para la comparativa
levels = ['Baja', 'Media', 'Alta']

def load_rgb(path):
    """Carga una imagen en formato RGB. Si falla, devuelve una imagen negra diminuta."""
    if not os.path.exists(path): return np.zeros((10,10,3), dtype=np.uint8)
    img = cv2.imread(path)
    if img is None: return np.zeros((10,10,3), dtype=np.uint8)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def show_results(img_name, original_path, filtered_paths):
    """Muestra los resultados en una matriz de 3x3 con la original en el centro superior."""
    fig, axes = plt.subplots(3, 3, figsize=(20, 18))
    
    # Ocultar todos los ejes inicialmente
    for row in axes:
        for ax in row:
            ax.axis('off')
    
    # Fila 1, Columna 2: Imagen Original Centrada
    axes[0, 1].imshow(load_rgb(original_path))
    axes[0, 1].set_title(f"--- ORIGINAL ({img_name}) ---", fontsize=14, fontweight='bold')
    axes[0, 1].axis('on')
    axes[0, 1].set_xticks([]); axes[0, 1].set_yticks([])
    
    # Fila 2: Resultados de NLM (Intensidad Baja, Media, Alta)
    for i in range(3):
        axes[1, i].imshow(load_rgb(filtered_paths[i]))
        axes[1, i].set_title(f"NLM - Intensidad {levels[i]}", fontsize=12)
        axes[1, i].axis('on')
        axes[1, i].set_xticks([]); axes[1, i].set_yticks([])
        
    # Fila 3: Resultados de BM3D (Intensidad Baja, Media, Alta)
    for i in range(3):
        axes[2, i].imshow(load_rgb(filtered_paths[i+3]))
        axes[2, i].set_title(f"BM3D - Intensidad {levels[i]}", fontsize=12)
        axes[2, i].axis('on')
        axes[2, i].set_xticks([]); axes[2, i].set_yticks([])
        
    plt.tight_layout()
    plt.suptitle(f"Matriz de Comparación: {img_name}", fontsize=18, y=1.02)
    plt.show()

## Métodos de Reducción de Ruido y Parámetros Disponibles

### 1. Non-Local Means (NLM)
El filtro **NLM** promedia píxeles basándose en la similitud de regiones (patches) en lugar de solo en la proximidad física. Esto permite una excelente conservación de bordes.

**Todos los parámetros disponibles:**
- `-h` o `--h`: **Fuerza del filtro**. Determina cuánto ruido se elimina. Valores recomendados: 5 a 30.
- `-t` o `--template_window_size`: **Tamaño del parche de plantilla**. Es el tamaño del bloque que se usa para comparar similitudes (debe ser impar, por defecto 7).
- `-s` o `--search_window_size`: **Tamaño del área de búsqueda**. Es la zona donde se buscan parches similares (debe ser impar, por defecto 21).

### 2. Block-Matching and 3D Filtering (BM3D)
**BM3D** utiliza el "filtrado colaborativo" agrupando bloques similares en 3D para separar el ruido de la señal original en el dominio de la transformada.

**Todos los parámetros disponibles:**
- `-s_bm3d` o `--sigma_bm3d`: **Desviación estándar del ruido**. Es el parámetro principal de intensidad.
- `-p_bm3d` o `--profile_bm3d`: **Perfil de ejecución**. Permite elegir entre:
    - `np`: Perfil Normal (predeterminado).
    - `lc`: Baja Complejidad (más rápido).
    - `high`: Alta Calidad (más lento).
    - `vn`: Ruido Extremo (para varianzas muy altas).
- `-psd_bm3d` o `--psd_bm3d`: **Densidad Espectral de Potencia**. Permite pasar un archivo `.npy` si el ruido no es blanco (ruido correlacionado).
- `-h_bm3d` o `--psf_bm3d`: **Función de Dispersión de Punto (PSF)**. Si se proporciona un archivo `.npy`, el algoritmo aplica **deblurring** junto con el denoising.

--- 
## Caso 1: Imagen `ruido.jpg`

In [ ]:
# 1. Codificar imagen original
!python ../src/no_filter.py encode -i ../img/ruido.jpg -e ../img/ruido_encoded

# 2. Decodificar aplicando NLM (h=5, 15, 30)
!python ../src/NLM.py decode -i ../img/ruido_encoded.tif -o ../img/nlm_h5.png --h 5
!python ../src/NLM.py decode -i ../img/ruido_encoded.tif -o ../img/nlm_h15.png --h 15
!python ../src/NLM.py decode -i ../img/ruido_encoded.tif -o ../img/nlm_h30.png --h 30

# 3. Decodificar aplicando BM3D (sigma=15, 30, 50)
!python ../src/BM3D.py decode -i ../img/ruido_encoded.tif -o ../img/bm3d_s15.png --sigma_bm3d 15.0
!python ../src/BM3D.py decode -i ../img/ruido_encoded.tif -o ../img/bm3d_s30.png --sigma_bm3d 30.0
!python ../src/BM3D.py decode -i ../img/ruido_encoded.tif -o ../img/bm3d_s50.png --sigma_bm3d 50.0

In [ ]:
paths_ruido = ['../img/nlm_h5.png', '../img/nlm_h15.png', '../img/nlm_h30.png', 
               '../img/bm3d_s15.png', '../img/bm3d_s30.png', '../img/bm3d_s50.png']
show_results('ruido.jpg', '../img/ruido.jpg', paths_ruido)

--- 
## Caso 2: Imagen `hombre.jpg`

In [ ]:
# 1. Codificar imagen original
!python ../src/no_filter.py encode -i ../img/hombre.jpg -e ../img/hombre_encoded

# 2. Decodificar aplicando NLM (h=5, 15, 30)
!python ../src/NLM.py decode -i ../img/hombre_encoded.tif -o ../img/hombre_nlm_h5.png --h 5
!python ../src/NLM.py decode -i ../img/hombre_encoded.tif -o ../img/hombre_nlm_h15.png --h 15
!python ../src/NLM.py decode -i ../img/hombre_encoded.tif -o ../img/hombre_nlm_h30.png --h 30

# 3. Decodificar aplicando BM3D (sigma=15, 30, 50)
!python ../src/BM3D.py decode -i ../img/hombre_encoded.tif -o ../img/hombre_bm3d_s15.png --sigma_bm3d 15.0
!python ../src/BM3D.py decode -i ../img/hombre_encoded.tif -o ../img/hombre_bm3d_s30.png --sigma_bm3d 30.0
!python ../src/BM3D.py decode -i ../img/hombre_encoded.tif -o ../img/hombre_bm3d_s50.png --sigma_bm3d 50.0

In [ ]:
paths_hombre = ['../img/hombre_nlm_h5.png', '../img/hombre_nlm_h15.png', '../img/hombre_nlm_h30.png', 
                '../img/hombre_bm3d_s15.png', '../img/hombre_bm3d_s30.png', '../img/hombre_bm3d_s50.png']
show_results('hombre.jpg', '../img/hombre.jpg', paths_hombre)